In [ ]:
# Auto-reload frequently changed files
%load_ext autoreload
%autoreload 2
%aimport utils

import pandas as pd
import numpy as np
import altair as alt
from ipywidgets import interact
from os.path import join

from constants import COLUMNS
from utils import read_latest_labs_df, read_loinc_df, apply_theme, apply_trellis_theme

alt.data_transformers.disable_max_rows();

# Dataset

In [ ]:
df = read_latest_labs_df()
SITE_IDS = df[COLUMNS.SITE_ID].unique().tolist()

loinc_df = read_loinc_df().set_index('loinc').rename(columns={'labTest': 'name'})

df["loinc_name"] = df[COLUMNS.LOINC].apply(lambda code: loinc_df.at[code, "name"].capitalize())

df.loc[df[COLUMNS.NUM_PATIENTS] < 0, COLUMNS.NUM_PATIENTS] = 0

df

# Visualizations

In [ ]:
# All vs individual
# independent vs shared

def render_chart(SiteID):
    line = alt.Chart(df).mark_line(size=1, opacity=1).encode(
        x=alt.X(
            f"{COLUMNS.DAYS_SINCE_POSITIVE}:Q", 
            title=None, 
            axis=alt.Axis(
                grid=True,
                labelOpacity=0, tickOpacity=0
            )
        ),
        y=alt.Y(
            f"mean({COLUMNS.MEAN_VALUE})", 
            title=None, 
            axis=alt.Axis(tickCount=2, orient="right")
        ),
        color=alt.Color("loinc_name:N", scale=alt.Scale(scheme="category20"), legend=None),
    ).properties(height=30, width=500)
    
    if SiteID != "All Sites":
        line = line.transform_filter(
            alt.FieldEqualPredicate(field="siteid", equal=SiteID)
        )
        
    area = line.mark_area(opacity=0.65)
    circle = line.mark_circle(size=0)
    
    top_chart = (circle + line + area).facet(
        row=alt.Row(
            "loinc_name:N",
            header=alt.Header(labelAngle=0, labelAlign="left", labelAnchor="middle", labelColor="black", title=None)
        ),
    ).resolve_scale(y="independent")

    bottom_chart = line.mark_bar().encode(
        y=alt.Y(
            "sum(num_patients)", 
            title="Number of tested patients", 
            axis=alt.Axis(
                tickCount=2, 
                titleAngle=0,
                titleAlign="right", 
                titleBaseline="middle",
                titlePadding=-545,
                orient="right"
            )
        ),
        x=alt.X(
            f"{COLUMNS.DAYS_SINCE_POSITIVE}:Q",
            bin=alt.Bin(step=1),
            title="Days since positive",
            axis=alt.Axis(
                grid=True,
                labelExpr="abs(parseInt(datum.value)) % 2 == 1 ? null : datum.label"
            )
        ),
        color=alt.value("gray")
    ).properties(height=35)

    return apply_trellis_theme(
        top_chart & bottom_chart
    ).resolve_scale(y="independent", x="shared").properties(title="LOINC test results")

interact(render_chart, SiteID=["All Sites"] + SITE_IDS)

In [ ]:
def render_chart(SiteID):
    line = alt.Chart(df).mark_line(size=2, opacity=0.8).encode(
        x=alt.X(
            f"{COLUMNS.DAYS_SINCE_POSITIVE}:Q", 
            title=None, 
            axis=alt.Axis(
                grid=True,
                labelOpacity=0, tickOpacity=0
            )
        ),
        y=alt.Y(
            f"mean({COLUMNS.MEAN_VALUE})", 
            title=None, 
            axis=alt.Axis(tickCount=2, orient="right")
        ),
        color=alt.Color("siteid:N", scale=alt.Scale(scheme="category10", range=["#377FB8", "#CA2026"])),
    ).properties(height=30, width=500)
    
    if SiteID != "All Sites":
        line = line.transform_filter(
            alt.FieldEqualPredicate(field="siteid", equal=SiteID)
        )
        
    circle = line.mark_circle(size=20)
    
    top_chart = (circle + line).facet(
        row=alt.Row(
            "loinc_name:N",
            header=alt.Header(labelAngle=0, labelAlign="left", labelAnchor="middle", labelColor="black", title=None)
        ),
    ).resolve_scale(y="independent")

    bottom_chart = line.mark_bar().encode(
        y=alt.Y(
            "sum(num_patients)", 
            title="Number of tested patients", 
            axis=alt.Axis(
                tickCount=2, 
                titleAngle=0,
                titleAlign="right", 
                titleBaseline="middle",
                titlePadding=-545,
                orient="right"
            )
        ),
        x=alt.X(
            f"{COLUMNS.DAYS_SINCE_POSITIVE}:Q",
            bin=alt.Bin(step=1),
            title="Days since positive",
            axis=alt.Axis(
                grid=True,
                labelExpr="abs(parseInt(datum.value)) % 2 == 1 ? null : datum.label"
            )
        ),
        color=alt.value("gray")
    ).properties(height=35)

    return apply_trellis_theme(
        top_chart & bottom_chart
    ).resolve_scale(y="independent", x="shared").properties(title="LOINC test results")

interact(render_chart, SiteID=["All Sites"] + SITE_IDS)

In [ ]:
line = alt.Chart(df).mark_line().encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive"),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)

point = alt.Chart(df).mark_point(filled=True).encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive"),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)


band = alt.Chart(df).mark_errorbar().encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive"),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    yError=alt.YError(COLUMNS.STDEV_VALUE),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)

(band + point + line).properties(
    width=150,
    height=150
).facet(
    facet=alt.Facet("loinc_name", title="LOINC"),
    columns=3
).resolve_scale(y="independent", x="shared")